# Week 1 Exercise | Study Guide Generation with Llama 3.2

In [ ]:
import requests
import json
import re
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = 'llama3.2'

### 1. Web Scraper

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

### 2. Curriculum Extraction

In [ ]:
curriculum_system_prompt = """You are provided with the text content of a webpage. 
Your task is to design a student-friendly curriculum from this content. 
Break down the material into clear modules or lessons, each with a title and a short description. 
Focus on organizing the information in a logical order, as if preparing a study plan.

You should respond in JSON as in this example:
{
    "curriculum": [
        {
            "module": "Introduction to Machine Learning",
            "description": "Basic concepts and history of machine learning, why it matters, and common applications."
        },
        {
            "module": "Supervised Learning",
            "description": "Learn about labeled data, classification, and regression methods."
        },
        {
            "module": "Unsupervised Learning",
            "description": "Understand clustering, dimensionality reduction, and when to use unsupervised approaches."
        }
    ]
}
"""

In [ ]:
def get_curriculum_user_prompt(website):
    user_prompt = f"Here is the text content of the website at {website.url}:\n\n"
    user_prompt += website.text
    user_prompt += "\n\nPlease create a student-friendly curriculum from this content. "
    user_prompt += "Break it down into clear modules or lessons, each with a title and a short description. "
    user_prompt += "Return your response in JSON format"
    return user_prompt

In [ ]:
def get_curriculum(website):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": curriculum_system_prompt},
            {"role": "user", "content": get_curriculum_user_prompt(website)}
        ],
        stream=True
    )
    response_text = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ''
        response_text += delta
        update_display(Markdown(response_text), display_id=display_handle.display_id)
    try:
        json_text = re.search(r"\{.*\}", response_text, re.DOTALL).group()
        curriculum_json = json.loads(json_text)
    except Exception as e:
        print("Failed to parse JSON:", e)
        curriculum_json = {"error": "JSON parse failed", "raw": response_text}

    return curriculum_json

### 3. Study Guide

In [ ]:
guide_system_prompt = """You are an educational assistant. 
You are given a curriculum JSON with modules and descriptions.
Your task is to create a student-friendly study guide based on this curriculum.
- Organize the guide step by step, with clear headings, tips, and examples where appropriate.
- Make it engaging and easy to follow.
- Adapt the content according to the student's level, language, and tone.
- Always respond in markdown format suitable for a student guide.
"""

In [ ]:
def get_study_guide_user_prompt(curriculum_json, student_level="beginner", language="English", tone="friendly"):
    return f"""
            Student Level: {student_level}
            Language: {language}
            Tone: {tone}
            
            Here is the curriculum JSON:
            
            {json.dumps(curriculum_json, indent=2)}
            
            Please convert it into a study guide for the student.
            """

In [ ]:
def stream_study_guide(curriculum_json, student_level="beginner", language="English", tone="friendly"):
    
    user_prompt = get_study_guide_user_prompt(curriculum_json, student_level, language, tone)
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": guide_system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
    )

    response_text = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ''
        response_text += delta
        update_display(Markdown(response_text), display_id=display_handle.display_id)
    
    return response_text

In [ ]:
page = Website("https://en.wikipedia.org/wiki/Rock_and_roll")
curriculum_json = get_curriculum(page)

In [ ]:
study_guide_text = stream_study_guide(
    curriculum_json,
    student_level="beginner",
    language="English",
    tone="friendly"
)

In [ ]:
study_guide_text = stream_study_guide(
    curriculum_json,
    student_level="advanced",
    language="English",
    tone="professional, detailed"
)